In [1]:
import unicodecsv
import pandas as pd
#read data from CSV file and make it to lists
def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
    
        return list(reader)
    
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

In [2]:
data_all = pd.read_csv('../../../2_data/explored/customer.csv')
customer_all=data_all['CUSTOMER_ID']
df_customer =pd.DataFrame(customer_all)
df_customer[:5]

,CUSTOMER_ID
0,2083
1,2102
2,2223
3,2224
4,2284


In [3]:
data_age = pd.read_csv('../../../2_data/explored/customer_age.csv')
order_item = pd.read_csv('../../../2_data/explored/order_item.csv')
order = pd.read_csv('../../../2_data/explored/order.csv')
order_succsess =pd.read_csv('../../../2_data/explored/order_success.csv')
product = pd.read_csv('../../../2_data/explored/product.csv')



goods = product['MATNR']
code=product['MAKTX']
df_goods =pd.DataFrame(goods)
df_code=pd.DataFrame(code)

df_product = pd.concat([df_goods,df_code], axis=1)
merge_id=pd.merge(order_succsess,order_item, on='ORDER_ID')
merge_product=pd.merge(product,merge_id, on='MATNR')



id_cus = merge_product['CUSTOMER_ID']
q=merge_product['QUANTITY']
matnr =merge_product['MATNR']
product_name = merge_product['MAKTX']



#oreder  success




# sum quantity

In [4]:
df2 = pd.concat([id_cus,matnr,q,product_name], axis=1, join='outer')


In [5]:
sum_df1 = df2.groupby(['CUSTOMER_ID','MATNR','MAKTX'])['QUANTITY'].sum().reset_index()
sum_df1.head()


,CUSTOMER_ID,MATNR,MAKTX,QUANTITY
0,21,25017251,SBDS ภาพแขวน #XC55153/XC,2
1,1042,25014525,SBDS เทียนตกแต่ง #5018N/YK,1
2,2082,19086644,MELONA/ตู้ผ้าWV137/DM01/D05LR/ขาว,1
3,2082,19086651,MELONA/โต๊ะทำงานDK120/ขาว,1
4,2082,19113824,BENTE/ตู้เก็บของC60/โซลิด/เดนิม-เทาดำ,1


# remove duplicate

In [6]:
df_id_remove = pd.DataFrame(sum_df1['CUSTOMER_ID'])

no_dup_id = df_id_remove.drop_duplicates(subset=['CUSTOMER_ID'])
no_dup_id[:5]

,CUSTOMER_ID
0,21
1,1042
2,2082
13,2391
14,2463


# setup index

In [7]:
no_dup_id = no_dup_id.reset_index()
no_dup_id['NEW_ID'] = no_dup_id.index + 1
df_new_id = pd.DataFrame(no_dup_id['NEW_ID']) 
df_new_id[:5]

,NEW_ID
0,1
1,2
2,3
3,4
4,5


In [8]:
add_new_id = pd.concat([no_dup_id], axis=1, join='outer')

In [9]:
add_new_id[:5]
del add_new_id['index']


In [10]:
add_new_id[:5]

,CUSTOMER_ID,NEW_ID
0,21,1
1,1042,2
2,2082,3
3,2391,4
4,2463,5


In [11]:
merge_sum_quan = pd.merge(add_new_id,sum_df1,how = 'left' , left_on =['CUSTOMER_ID'], right_on = ['CUSTOMER_ID'])
merge_sum_quan[:5]


,CUSTOMER_ID,NEW_ID,MATNR,MAKTX,QUANTITY
0,21,1,25017251,SBDS ภาพแขวน #XC55153/XC,2
1,1042,2,25014525,SBDS เทียนตกแต่ง #5018N/YK,1
2,2082,3,19086644,MELONA/ตู้ผ้าWV137/DM01/D05LR/ขาว,1
3,2082,3,19086651,MELONA/โต๊ะทำงานDK120/ขาว,1
4,2082,3,19113824,BENTE/ตู้เก็บของC60/โซลิด/เดนิม-เทาดำ,1


# new table

In [12]:
customer_id = merge_sum_quan['NEW_ID']
customer_id_old =merge_sum_quan['CUSTOMER_ID']
matnr1 =merge_sum_quan['MATNR']
quan = merge_sum_quan['QUANTITY']
sum_df2 = pd.concat([customer_id_old,customer_id,matnr1,quan], axis=1, join='outer')
sum_df2[:5]


,CUSTOMER_ID,NEW_ID,MATNR,QUANTITY
0,21,1,25017251,2
1,1042,2,25014525,1
2,2082,3,19086644,1
3,2082,3,19086651,1
4,2082,3,19113824,1


# change matnr to string

In [13]:
str_matnr =sum_df2['MATNR'].apply(str)

In [14]:
sum_df = pd.concat([customer_id_old,customer_id,str_matnr,quan], axis=1, join='outer')

In [15]:
sum_df[:5]

,CUSTOMER_ID,NEW_ID,MATNR,QUANTITY
0,21,1,25017251,2
1,1042,2,25014525,1
2,2082,3,19086644,1
3,2082,3,19086651,1
4,2082,3,19113824,1


In [16]:
sum_df['CUSTOMER_ID'] = sum_df['CUSTOMER_ID'].astype(int).astype(str)


In [17]:
sum_df.dtypes

CUSTOMER_ID    object
NEW_ID          int64
MATNR          object
QUANTITY        int64
dtype: object

In [18]:
np.unique(sum_df['QUANTITY'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 18, 24, 39])

# dataframe of goods

In [19]:
matnr2 = df2['MATNR']
maktx= df2['MAKTX']
df_goods = pd.concat([matnr2,maktx], axis=1, join='outer')
df_goods[:5]

,MATNR,MAKTX
0,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
1,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
2,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
3,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
4,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว


# pivot table

In [45]:
pivot = sum_df.pivot_table(index='CUSTOMER_ID', columns='MATNR', values='QUANTITY', aggfunc=np.sum, fill_value = 0)
pivot.index.name = None
pivot.columns.name = None
pivot

,19000158,19000346,19000900,19005144,19008811,19008833,19009152,19009245,19011555,19012447,...,59015400,59015657,59015667,59015674,59015710,59015711,59015916,59016460,59016463,59016513
100085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100683,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100684,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100806,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100822,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100843,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100982,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# กำหนดให้ null = 0

In [46]:
pivot.fillna( 0, inplace = True )

In [47]:
pivot.shape

(3388, 2351)

In [48]:
pivot.max()

19000158    1
19000346    1
19000900    1
19005144    2
19008811    1
19008833    1
19009152    1
19009245    1
19011555    1
19012447    1
19012992    1
19013191    1
19014268    1
19014291    1
19014293    1
19014714    1
19015027    1
19015840    2
19015844    1
19015920    1
19016097    2
19022748    1
19023000    1
19023001    1
19023002    1
19023015    1
19023021    1
19023171    8
19023350    4
19023360    1
           ..
59014420    1
59014421    2
59014422    1
59014423    2
59014424    1
59014440    1
59014544    1
59014712    1
59014907    1
59014981    1
59015236    1
59015237    1
59015246    1
59015249    1
59015386    1
59015387    1
59015389    1
59015390    1
59015391    1
59015392    1
59015400    1
59015657    1
59015667    1
59015674    1
59015710    1
59015711    1
59015916    1
59016460    1
59016463    1
59016513    1
Length: 2351, dtype: int64

# Calculate the similar

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
user_sim = cosine_similarity(pivot.as_matrix())
user_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [50]:
user_sim.max()

1.0000000000000002

In [51]:
np.fill_diagonal( user_sim, 0)

In [52]:
user_sim.max()

1.0

In [53]:
df_corr = pd.DataFrame(user_sim, columns=pivot.index.values, index=pivot.index)
df_corr

,100085,100522,100622,100683,100684,100806,100822,100843,100862,100982,...,99082,99087,99222,99582,99702,99782,99885,99887,99923,99946
100085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100683,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
100982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


# sort correlation

In [54]:
-np.sort(-df_corr[:1])

array([[0.5, 0. , 0. , ..., 0. , 0. , 0. ]])

In [59]:
sort_recommend = df_corr.sort_values('99885', ascending=False)
sort_recommend['99885'][:5]

82567     0.707107
101543    0.707107
105764    0.707107
202062    0.707107
30503     0.707107
Name: 99885, dtype: float64

# test 2

In [33]:
user_sim

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
def get_position(M):
    row_columns = []
    cols = M.columns
    
    for i in range(0, M.shape[1]):
        for j in range(0, i+1):
            row_columns.append((cols[i], cols[j]))
            return row_columns

In [42]:
get_position(pivot)

[('100085', '100085')]

In [43]:
def get_rec(id_user,M,num):
    import numpy as np
    reviews = []
    cols = get_position(M)

    M = user_sim  
    for cor in M:
        if cor == id_user:
            return cor
        else:
            reviews.append((cols,cor))
            reviews.sort(key=lambda tup:[1],reverse=True)
            return reviews[:num]

In [44]:
rec = get_rec('2082',pivot,3)
rec

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


[([('100085', '100085')], array([0., 0., 0., ..., 0., 0., 0.]))]

In [46]:
user_sim_df.iloc[519:530, 519:530]

,519,520,521,522,523,524,525,526,527,528,529
519,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
520,0.0,1.000000,0.182574,0.408248,0.204124,0.0,0.0,0.0,0.0,0.0,0.0
521,0.0,0.182574,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
522,0.0,0.408248,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
523,0.0,0.204124,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0
524,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
525,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
526,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0
527,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
528,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0


# similar

In [70]:
user_sim_df.idxmax(axis=1)[0:5]

0    0
1    1
2    2
3    3
4    4
dtype: int64

# Setting correlation with self to 0

In [71]:
np.fill_diagonal( user_sim, 0 )

In [72]:
user_sim_df = pd.DataFrame( user_sim )

In [73]:
user_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Top  correlation

ดึงแถวที่อยู่ใน pivot

In [34]:
user = user_sim_df[519]


In [35]:
max_corr = -np.sort(-user)
max_corr

array([0.4472136, 0.4472136, 0.4472136, ..., 0.       , 0.       ,
       0.       ])

In [36]:
max_corr.shape

(3388,)

# สร้าง dataframe ใหม่

In [37]:
df_u1 = pd.DataFrame(max_corr)
df_u1[:5]

,0
0,0.447214
1,0.447214
2,0.447214
3,0.447214
4,0.447214


เริ่มเปลี่ยน correlationจาก 0.447214 ตั้งแต่indexที่17

In [41]:
sort_top = -np.sort(-user_sim_df)
sort_top

array([[0.39223227, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.63245553, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.21320072, 0.21320072, 0.19069252, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.89442719, 0.89442719, 0.89442719, ..., 0.        , 0.        ,
        0.        ],
       [0.70710678, 0.57735027, 0.40824829, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

# sort top correlation

In [55]:
sort_top_recommend = pd.DataFrame(sort_top)

In [43]:
sort_top.shape

(3388, 3388)

In [58]:
sort_top_recommend.head()

,0,1,2,3,4,5,6,7,8,9,...,3378,3379,3380,3381,3382,3383,3384,3385,3386,3387
0,0.392232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.632456,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.213201,0.213201,0.190693,0.190693,0.174078,0.150756,0.150756,0.150756,0.150756,0.150756,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.707107,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# correlation

In [44]:
user_sim_df.max()

0       0.392232
1       0.632456
2       0.213201
3       0.707107
4       1.000000
5       1.000000
6       1.000000
7       0.577350
8       1.000000
9       1.000000
10      0.000000
11      0.577350
12      0.263117
13      0.500000
14      1.000000
15      0.639602
16      1.000000
17      1.000000
18      1.000000
19      0.816497
20      0.816497
21      0.688247
22      0.894427
23      0.577350
24      0.707107
25      1.000000
26      1.000000
27      1.000000
28      0.670820
29      1.000000
          ...   
3358    0.000000
3359    0.707107
3360    1.000000
3361    0.408248
3362    1.000000
3363    0.000000
3364    0.707107
3365    1.000000
3366    0.577350
3367    0.000000
3368    0.707107
3369    1.000000
3370    0.577350
3371    0.000000
3372    0.707107
3373    0.707107
3374    1.000000
3375    1.000000
3376    0.000000
3377    0.707107
3378    1.000000
3379    1.000000
3380    0.707107
3381    1.000000
3382    0.000000
3383    0.000000
3384    0.970143
3385    0.8944

In [45]:
df_goods.loc[df_goods['MATNR'] == 19096167]

,MATNR,MAKTX
1351,19096167,ECONI/เตียงBW5'/C/CANYON/I-TWIST/ผ้าเทา
1352,19096167,ECONI/เตียงBW5'/C/CANYON/I-TWIST/ผ้าเทา
1353,19096167,ECONI/เตียงBW5'/C/CANYON/I-TWIST/ผ้าเทา
1354,19096167,ECONI/เตียงBW5'/C/CANYON/I-TWIST/ผ้าเทา
1355,19096167,ECONI/เตียงBW5'/C/CANYON/I-TWIST/ผ้าเทา
1356,19096167,ECONI/เตียงBW5'/C/CANYON/I-TWIST/ผ้าเทา
1357,19096167,ECONI/เตียงBW5'/C/CANYON/I-TWIST/ผ้าเทา


# Finding user similarities max

In [46]:
user_sim_df.idxmax(axis=1).sample( 10, random_state = 10 )

3218     249
1029       0
2451    2778
2497    1925
85      1096
1523       0
1893    2489
510      539
1046       0
969     1084
dtype: int64

In [47]:
df_not_dup = df_goods.drop_duplicates(subset=['MATNR'])
df_not_dup[:5]

,MATNR,MAKTX
0,19047936,PATISIA/ตู้ข้างเตียงNT45/ขาว
7,59001810,Bedroom/ALicia-E 6'&Meudon/White/Re
8,19047934,PATISIA/ตู้สูงSC90/ขาว
9,19031688,EXIO/กระจกแขวนEM120/ขาว/เงา
10,19035027,EXIO/เตียง3.5'/A/ขาAเงา/ดำ/ขาว/ขาว


In [48]:
sum_df1.loc[sum_df1['CUSTOMER_ID'] == 2082]


,CUSTOMER_ID,MATNR,MAKTX,QUANTITY
2,2082,19086644,MELONA/ตู้ผ้าWV137/DM01/D05LR/ขาว,1
3,2082,19086651,MELONA/โต๊ะทำงานDK120/ขาว,1
4,2082,19113824,BENTE/ตู้เก็บของC60/โซลิด/เดนิม-เทาดำ,1
5,2082,25014334,VP กล่องเอนกประสงค์ #HS-13MG070/HS,1
6,2082,25025165,หมอน45X45/LEAVE YOUR MESSAGE,1
7,2082,25025166,หมอน45X45/TAKE A BREAK,1
8,2082,25025851,SBหมอนอิง#1639B-02-017/ผ้า/แมว/HMH,1
9,2082,25025861,SBหมอนอิง#137B-083-2-D/ผ้า/เทาเข้ม/HMH,3
10,2082,25027036,หมอน45X45/ลายBELIEVE,1
11,2082,25027768,หมอน45X45/IF YOU LOVE ME LET ME SLEEP,2


In [49]:
df_not_dup.loc[df_not_dup['MATNR'] == 19096167]

,MATNR,MAKTX
1351,19096167,ECONI/เตียงBW5'/C/CANYON/I-TWIST/ผ้าเทา


In [50]:
def get_user_similar_movies(user1, user2):
    common_movies = sum_df[sum_df.NEW_ID == user1].merge(
      sum_df[sum_df.NEW_ID == user2],
      on = "MATNR",
      how = "inner" )

    return common_movies.merge(df_not_dup, on = 'MATNR' )

# ลองทดสอบ

In [72]:
common_movies = sum_df[sum_df.NEW_ID == 3218].merge(sum_df[sum_df.NEW_ID == 249],on = "MATNR",how = "inner")
#result = pd.concat([df1, df4], axis=1, join='inner')

common_movies.merge(df_goods, on = 'MATNR' )

,CUSTOMER_ID_x,NEW_ID_x,QUANTITY_x,CUSTOMER_ID_y,NEW_ID_y,QUANTITY_y,MATNR,MAKTX


In [71]:
get_user_similar_movies(2452,2779)

,CUSTOMER_ID_x,NEW_ID_x,MATNR,QUANTITY_x,CUSTOMER_ID_y,NEW_ID_y,QUANTITY_y,MAKTX


# test

In [25]:
def get_recs(id_user,M,num):
    import numpy as np
    
    reviews = []
    for title in M.columns:
        if title == id_user:
            continue
        cor = peason(pivot[id_user],pivot[title])
        if np.isnan(cor):
            continue
        else:
            reviews.append((title,cor))
            reviews.sort(key=lambda tup:[1],reverse=True)
            return reviews[:num]
                

In [26]:
recs = get_recs('21',pivot,10)
recs

[('100085', -0.0004255092820230912)]